# Differential operators as matrices

Differential operators are linear operators on vector fields of functions. They have many similar properties to matrices on finite-dimensional vector spaces. Indeed, if a function $y(x)$ is sampled at discrete points with values $y_n$, then the approximations to $y'$, $y''$ are created by multiplying the vector $(y_n)$ with a difference matrix.

In particular, the eigenvalues and eigenfunctions of an operator can be approximated by looking at the eigenvalues and eigenvectors of the discretised matrix operator. This can be done numerically using methods such as `eigh` from the `scipy` package: see [its documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html) for information about its algorithm. (We are only interested in self-adjoint operators. When discretised, they become Hermitian matrices.) Note that most such algorithms return the eigenvectors as normalised vectors; their actual magnitude when interpreted as functions needs to be scaled by $\sqrt{N}$, where $N$ is the number of gridpoints.

In [364]:
import numpy as np
from numpy import pi
from numpy import sqrt, sin, cos, tan, sinh, cosh, exp, dot
# from numpy.linalg import eig, eigh
from scipy.linalg import eig, eigh
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [331]:
def discretise(xa, xb, nx=101):
    """Returns the xs, the spacing dx between the xs,
    and the identity and first and second derivative
    matrices, computed using a 2nd-order scheme."""
    xs = np.linspace(xa, xb, nx)
    dx = xs[1] - xs[0]

    eye = np.eye(nx)
    
    ddx = (np.roll(eye, 1, 1) - np.roll(eye, -1, 1)) / (2*dx)
    # Use 3rd-order scheme at boundaries?
    ddx[0, :] = 0; ddx[0, :4] = (-11/6, 3, -3/2, 1/3) / dx
    ddx[-1, :] = 0; ddx[-1, -4:] = (-1/3, 3/2, -3, 11/6) / dx

    d2dx2 = (np.roll(eye, 1, 1) - 2*eye + np.roll(eye, -1, 1)) / (dx*dx)
    d2dx2[0, :] = 0; d2dx2[0, :4] = (2, -5, 4, -1) / (dx*dx)
    d2dx2[-1, :] = 0; d2dx2[-1, -4:] = (-1, 4, -5, 2) / (dx*dx)
    
    return xs, dx, eye, ddx, d2dx2

## The harmonic functions
These are the eigenfunctions of the operator $-D^2$.

In [334]:
xa = 0; xb = pi; nx = 101;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
eigvals, eigvecs = eigh(-d2dx2)
@interact(mode=widgets.IntSlider(min=0, max=12))
def harmfun(mode):
    plt.plot(xs, eigvecs[:, mode+2] * sqrt(nx)) # TODO what's with modes 0 and 1?
    plt.gca().set(title=f"E = %.2f" % eigvals[mode+2], 
                  xlim=[xa, xb], ylim=[-2, 2])
    plt.gca().grid()

interactive(children=(IntSlider(value=0, description='mode', max=12), Output()), _dom_classes=('widget-interac…

## The quantum harmonic oscillator

In suitable units, the time-independent Schrodinger equation for the wavefunction in a harmonic potential is 
$$ -y'' + x^2 y = E y.$$
The possible energy levels $E$ are the eigenvalues of the operator $-D^2 + x^2$. The operator 'multiply by $x^2$' is represented as a diagonal matrix whose elements are $x_n^2$, where $x_n$ are the gridpoints of the discretisation.

In [333]:
xa = -10; xb = 10; nx = 501;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d2dx2[0, :] = 0; d2dx2[-1, :] = 0
eigvals, eigvecs = eigh(-d2dx2 + np.diag(xs**2))
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=0, max=12))
def qhoplot(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs, eigvecs[:, mode+1])
    plt.gca().set(title=f"E = %.2f" % eigvals[mode+1], 
                  xlim=[-10, 10], ylim=[-.16, .16])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

interactive(children=(IntSlider(value=0, description='mode', max=12), Output()), _dom_classes=('widget-interac…

## Sheet 1, question 7

$$ y'' + 4y' + 4y = -\lambda y $$

so the operator of interest is $D^2 + 4D + 4I$. Note that this is _not_ a self-adjoint operator, so we'll need to use the `eig` solver, not `eigh`.

In [404]:
xa = 0; xb = 1; nx = 301;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d2dx2[0, :] = 0; d2dx2[-1, :] = 0

In [414]:
eigvals, eigvecs = eig(d2dx2 + 4*ddx + 4*eye)
eigvals = -eigvals
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=0, max=12))
def s1q7_plot_notsl(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs[0::2], eigvecs[0::2, mode+1] * sqrt(nx) * np.sign(eigvecs[2, mode+1]))
    plt.gca().set(title=f"E = %.2f" % eigvals[-mode-2], 
                  xlim=[0, 1], ylim=[-3, 3])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

interactive(children=(IntSlider(value=0, description='mode', max=12), Output()), _dom_classes=('widget-interac…

We can turn the equation into a self-adjoint equation by multiplying through by the integrating factor $p(x) = \exp(4x)$, to get

$$ (py')' + 4py = -\lambda py. $$

As a matrix equation, this becomes

$$ 
(DPD + 4P)y = -\lambda Py,
$$
a generalised eigenvector problem. As in the QHO example, $P$ is a diagonal matrix whose elements are $p(x_n) = \exp(4x_n)$, representing multiplication by $p(x)$. So, recasting in Sturm-Liouville form is akin to diagonalising a matrix.

In [409]:
p_mat = np.diag(exp(4*xs))
eigvals, eigvecs = eigh(ddx @ p_mat @ ddx + 4 * p_mat, p_mat)
eigvals = -eigvals

In [413]:
@interact(mode=widgets.IntSlider(min=0, max=12))
def s1q7_plot_sl(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs[0::4], eigvecs[0::4, 2*mode+2] * sqrt(nx) * np.sign(eigvecs[4, 2*mode+2]))
    plt.gca().set(title=f"E = %.2f" % eigvals[-2*mode-4], 
                  xlim=[0, 1], ylim=[-1, 1])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

interactive(children=(IntSlider(value=0, description='mode', max=12), Output()), _dom_classes=('widget-interac…

## Sheet 1, question 8: Bessel functions

In [442]:
xa = 0.01; xb = 10; nx = 601;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d2dx2[0, :] = 0; d2dx2[-1, :] = 0

In [444]:
x_mat = np.diag(xs)
eigvals, eigvecs = eigh(ddx @ x_mat @ ddx, x_mat)
# eigvals = -eigvals
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=0, max=12))
def s1q8_plot_notsl(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs, eigvecs[:, mode] * np.sign(eigvecs[0, mode]))
    plt.gca().set(title=f"E = %.2f" % eigvals[-mode], 
                  xlim=[xa, xb], ylim=[-.25, .25])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

interactive(children=(IntSlider(value=0, description='mode', max=12), Output()), _dom_classes=('widget-interac…

In [430]:
eigvals

array([           -inf-0.j, -3.92804779e-02-0.j,  4.03231911e-02-0.j,
        3.13207410e+00-0.j,  9.45081084e+00-0.j,  2.19009240e+01-0.j,
        3.94406546e+01-0.j,  6.07874639e+01-0.j,  8.92554488e+01-0.j,
        1.19185856e+02-0.j,  1.58947878e+02-0.j,  1.97039118e+02-0.j,
        2.48498212e+02-0.j,  2.94313828e+02-0.j,  3.57869398e+02-0.j,
        4.10975220e+02-0.j,  4.87012849e+02-0.j,  5.46983003e+02-0.j,
        6.35869855e+02-0.j,  7.02290208e+02-0.j,  8.04372070e+02-0.j,
        8.76842718e+02-0.j,  9.92441745e+02-0.j,  1.07057905e+03-0.j,
        1.19999186e+03-0.j,  1.28343022e+03-0.j,  1.42692622e+03-0.j,
        1.51531961e+03-0.j,  1.67313955e+03-0.j,  1.76616291e+03-0.j,
        1.93851757e+03-0.j,  2.03586807e+03-0.j,  2.22293708e+03-0.j,
        2.32433517e+03-0.j,  2.52626605e+03-0.j,  2.63145643e+03-0.j,
        2.84836371e+03-0.j,  2.95711609e+03-0.j,  3.18908066e+03-0.j,
        3.30119043e+03-0.j,  3.54825899e+03-0.j,  3.66354765e+03-0.j,
        3.92573242e+